In [27]:
from datasets import load_dataset
from gensim.utils import tokenize
from tqdm import tqdm
from textblob import TextBlob
data=load_dataset('imdb')

In [29]:
all_sentences=[]
threshold=32

for text_block in tqdm(data['train']['text']):
    for sentences in TextBlob(text_block).sentences:
        len_of_sent=sentences.words 
        for sent in sentences.split('.<br /><br />'):
            if len(len_of_sent)<threshold:
                all_sentences.append(sent)
    
    


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:22<00:00, 1118.44it/s]


In [35]:
all_sentences[500]

'I thought this was a very clunky, uninvolving version of a famous Australian story.'

In [12]:
all_texts=TextBlob(data['train']['text'][0])

In [14]:
all_texts.sentences

[Sentence("I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967."),
 Sentence("I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life."),
 Sentence("In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States."),
 Sentence("In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic."),
 Sente

In [53]:
b

['I',
 Sentence("I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967."),
 Sentence("I rented I of CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967.")]